# <center> <font color = 'rebeccapurple'>Stochastic Watershed Model: Generating Stochastic Streamflows for the Shasta River Under 3 Degrees of Warming</font> </center>

This Jupyter Notebook contains code for a stochastic watershed model (SWM) that can be used to develop an ensemble of stochastic streamflows for a single site. This code was used to produce the results found in **Shabestanipour et al. (2024)** for the Shasta River basin. With modifications to the input data and the specific ARMA model used (more details below), this code could be applied to other non-regulated river basins. 

This notebook is split into two parts: 1. SWM (generating stochastic streamflows) and 2. Diagnostics (assessing stochastic model performance).

***

**Computational Requirements**: We used Jupyter version 4.7.1 and Python version 3.8.8 to run the below code. Lower versions may result in error messages.

The code is somewhat computationally-intesnive. We found that allocating 32GB of memory allowed the code to run in full without crashing the kernel. 

The code blocks that will take the longest are generating the stochastic residuals in Part 1 and generating the figure inputs in Part 2 (depending on how many stochastic realizations you do). These should each take approx. 10 min maximum. Most other code blocks run very quickly. The code to generate AIC/BIC for various ARIMA models to determine which to use may also take some time (by default this code block is commented out).

***

# <center> <font color = 'red'> Part 1: SWM </font> </center>

## <font color='green'> Import Statements </font>

Here we import SWM functions (developed by Ghazal Shabestanipour) as well as existing external Python packages used in the notebook. Be sure that the file **Func_Lib.py** is contained in the same directory as this Jupyter Notebook.
For existing Python packages, if you do not have a particular package installed with your version of Python, you can install it using pip or conda (if using anaconda). Instructions for doing so can be found here:<br>
https://packaging.python.org/en/latest/tutorials/installing-packages/ (pip)<br>
https://docs.conda.io/projects/conda/en/latest/user-guide/concepts/installing-with-conda.html (conda)

In [1]:
# functions developed by Ghazal:
import Func_Lib_forAbby as func

# general packages
import numpy as np #numpy is a scientific computing package
import pandas as pd #pandas is a data analysis package
import math #math contains math functions
import time #functions in this package can be used to understand how long the code is taking to run
import os #functions related to operating system

# plotting packages
import matplotlib.pyplot as plt #matplotlib.pyplot is used for plotting
import seaborn as sns #seaborn is a plotting package
plt.style.use('seaborn-white') #this is the specific style of plotting used

# statistical analysis packages
from scipy import stats #scipy.stats are functions related to statistical analysis
from scipy.stats import norm #functions for normally distributed random variable 

# time series analysis packages
import statsmodels as sm
from statsmodels.graphics.tsaplots import plot_acf #function to plot autocorrelation function (acf) for time series analysis
from statsmodels.graphics.tsaplots import plot_pacf #function to plot partial autocorrelation function (pacf) for time series analysis
#from statsmodels.tsa.arima_model import ARIMA #import function for ARIMA (time series analysis)
from statsmodels.tsa.arima.model import ARIMA

## <font color='green'> Load Input Data </font>

The only input data required is a csv containing daily streamflow data (observed and modeled). The columns of the csv are $date$, observed streamflow $Qgage$, deterministic model of streamflow $Qmodel$, and the difference between Qgage and Qmodel for each date $diff$.

For the Squannacook, we have values from 10/01/1987 - 09/30/2018.

In [2]:
# read in the csv of streamflow data and assign to variable called "data"
data = pd.read_csv('SWM-data_cond-debias.csv') #columns are date, Qgage (observed), Qmodel (deterministic model), and diff (Qmodel - Qgage)

data['month'] = pd.DatetimeIndex(data['date']).month #create a DatetimeIndex column for month
data['date'] = pd.to_datetime(data['date']) #convert format of "date" column to Datetime (for time series analysis)

#convert units to m^3/s 
# mm/day * SHA area in mi2 * (1609.34^2 m^2/mi^2) * (10^-3 m/1 mm) * (1 day/24*3600 s)
area_mi2_SHA = 6665 #area for Shasta baisn in sq. mi. from Zach Brodeur
data['Qgage'] = data['Qgage']*area_mi2_SHA*(1609.34**2)*(10**-3)/(24*3600)
data['Qmodel'] = data['Qmodel']*area_mi2_SHA*(1609.34**2)*(10**-3)/(24*3600)

# generating empirical log-ratio errors
data['lambda'] = np.log(data['Qmodel'] / data['Qgage']) #calculate the lambda = log(Qmodel/Qgage)
data['raw diff'] = data['Qmodel'] - data['Qgage']
data #display first five rows of data

,date,Qgage,Qmodel,month,lambda,raw diff
0,1987-10-01,59.938142,366.261850,10,1.810035,306.323708
1,1987-10-02,99.896904,735.416652,10,1.996299,635.519749
2,1987-10-03,99.896904,550.188717,10,1.706123,450.291814
3,1987-10-04,79.917523,421.519228,10,1.662870,341.601705
4,1987-10-05,99.896904,336.630618,10,1.214848,236.733714
...,...,...,...,...,...,...
11318,2018-09-26,79.917523,78.229589,9,-0.021347,-1.687934
11319,2018-09-27,79.917523,77.855393,9,-0.026142,-2.062129
11320,2018-09-28,59.938142,77.513991,9,0.257145,17.575849
11321,2018-09-29,79.917523,81.235286,9,0.016355,1.317763


In [3]:
#load in a "future" scenario into a new datafrae df_fut
df_fut = pd.read_csv('SWM-data_cond-debias_scenario-13.csv') #make sure this is one of the five we want, I just used a random one

#put the future scenario into the correct units of m3/s
df_fut['Qfut'] = df_fut['Qsim_cbias']*area_mi2_SHA*(1609.34**2)*(10**-3)/(24*3600)
df_fut['date'] = pd.to_datetime(df_fut['date'])
df_fut = df_fut[(df_fut['date'] > '1987-09-30') & (df_fut['date'] < '2018-10-01')]
df_fut = df_fut.reset_index()
df_fut['index']=np.arange(0, 11323)

#df_fut = df_fut.iloc[0:11323]

df_fut

,index,date,Qsim,Qsim_cbias,Qfut
0,0,1987-10-01,0.008264,0.414617,82.837995
1,1,1987-10-02,0.008134,0.414027,82.720013
2,2,1987-10-03,0.007987,0.413357,82.586194
3,3,1987-10-04,0.007855,0.412756,82.466052
4,4,1987-10-05,0.007708,0.412078,82.330604
...,...,...,...,...,...
11318,11318,2018-09-26,0.011616,0.382011,76.323410
11319,11319,2018-09-27,0.011326,0.380637,76.048830
11320,11320,2018-09-28,0.011037,0.379260,75.773864
11321,11321,2018-09-29,0.013276,0.389679,77.855357


## <font color='green'> Fit ARMA Model to Data </font>

Here we fit an AR4 model to our "lambdas" to get the model residuals and parameters.

In [4]:
p=4 #order of AR (autocorrelation with lag 4)
d=0 #order of I (no trend)
q=0 #order of MA (no moving average component)

mod = ARIMA(data['lambda'], order=(p,d,q)) #set up AR4 model using lambdas as input
res = mod.fit() #fit model

In [5]:
res.params #display the model parameters

const     0.023440
ar.L1     0.165057
ar.L2     0.187447
ar.L3     0.147261
ar.L4     0.137038
sigma2    0.067909
dtype: float64

Since we fit an AR4 model for this data, we have five parameters: a coefficient for each AR component (L1, L2, L3, and L4) and a constant term.

In [6]:
#create a column in data for the model residuals associated with each time step
data['Ar_res']=res.resid
data.head(5)

,date,Qgage,Qmodel,month,lambda,raw diff,Ar_res
0,1987-10-01,59.938142,366.261850,10,1.810035,306.323708,1.786596
1,1987-10-02,99.896904,735.416652,10,1.996299,635.519749,1.411201
2,1987-10-03,99.896904,550.188717,10,1.706123,450.291814,0.761985
3,1987-10-04,79.917523,421.519228,10,1.662870,341.601705,0.583921
4,1987-10-05,99.896904,336.630618,10,1.214848,236.733714,0.070037


## <font color = 'green'> Generate Random AR Parameter Sets </font>

We generate $m$ random AR parameter sets (called "Beta") to consider the uncertainty in the AR model parameters.

We use the Numpy function **random.multivariate_normal** to generate the parameter sets. This function samples the parameters from a multivariate normal distribution. The inputs to the function are the mean **(res.params)**, the covariance **(res.cov_params)**, and the number of parameter sets to generate. 

In [7]:
# m is the number of random AR parameter sets, Beta, generated for considering AR model uncertainty.
m = 100

# older version of ARIMA function
#Beta=np.random.multivariate_normal(np.array(res.params), np.array(res.cov_params()), m)

# updated Beta for using newer version of statsmodels ARIMA code
# Beta=np.random.multivariate_normal(np.array(res.params[:-1]), np.array(res.cov_params())[:-1,:-1], m)

# for reproducible Betas, use scipy.stats multivariate_normal function instead, setting random seed
Beta = stats.multivariate_normal(np.array(res.params[:-1]),np.array(res.cov_params())[:-1,:-1], seed=16)
Beta = Beta.rvs(m)

## <font color = 'green'> Generate Stochastic Residuals Using Simple Bootstrap </font>

There are 3 steps to generating the stochastic streamflow realizations:<br>

1. Generate stochastic residuals (use either simple bootstrap, conditional bootstrap, or k-Nearest Neighbors bootstrap)
2. Generate stochastic log errors by combining AR model uncertainty + stochastic residuals in AR4 model
3. Generate stochastic streamflow by transforming from log-ratios back to streamflows, including bias correction factor<br>

### <font color = 'green'> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Step 1: Generate Stochastic Residuals <green>

In [8]:
#now find the nearest Qmodel and residual associated with Qmodel for each flow value in Qfut
df_fut['Qmodel_nearest'] = 0 #initialize empty column for nearest Qmodel
df_fut['residual_nearest'] = 0 #initialize empty column for nearest Qmodel residual

for i in range(len(df_fut)): #loop through each row of df_fut
    #for each flow, calculate absolute difference of that flow and all flows in Qmodel
    #take absolute value and then find the index of the minimum absolute value
    #populate Qmodel_nearest column with data['Qmodel'] at this index
    #and residual_nearest column with data['Ar_res'] at this index
    df_fut['Qmodel_nearest'].iloc[i] = data['Qmodel'].iloc[abs(data['Qmodel'] - df_fut['Qfut'].iloc[i]).idxmin()]
    df_fut['residual_nearest'].iloc[i] = data['Ar_res'].iloc[abs(data['Qmodel'] - df_fut['Qfut'].iloc[i]).idxmin()]
    
#now use KNN bootstrapping as before, with Epsilon defined as the Qmodel_nearest and residual_nearest columns from df_fut
#basically exactly what we had before, with some data "repeats" to correspond to the new data points
#and in the correct order to match Qfut

n = 100 # n is the number of stochastic streamflow realizations generated for each Beta set
Epsilon = df_fut.filter(['residual_nearest','Qmodel_nearest'])
Epsilon = np.array(Epsilon) # turn dataframe into numpy array

E = [] #initialize empty list to store errors
for t in range(0,len(df_fut)): # loop through all time periods
    E.append(Epsilon - [0,Epsilon[t,1]])  # first column residual, second column diff between flow and all other flows

k = 750

Er = np.zeros((len(df_fut),k)) #initialize empty 2d array with dimensions time periods x k (# nearest neighbors)
for t in range(0,len(df_fut)): #loop through all time periods
    E[t][:,1] = np.abs(E[t][:,1]) #get absolute values of errors
    x = E[t][np.argsort(E[t][:, 1])] #sort errors by magnitude
    Er[t,:] = x[0:k,0]
    

R = np.zeros((len(df_fut),n*m)) #initialize empty 2d array with dimensions time periods x # realizations
for i in range(0,n*m): #loop through all realizations
    I = np.random.randint(low=0, high=k, size=len(df_fut)) #randomly identify indices
    R[:,i] = Er[tuple(np.arange(0,len(df_fut),1)),I] #sample from Er at random indices I
    
E = R #set E to equal R

/cluster/tufts/hpc/tools/anaconda/202105/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


One needs to experiment with different k values to find the appropriate k for each basin by going trough all the next steps of the SWM and diagnostic figures.  

### <font color='green'> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Step 2: Generate Stochastic Log Errors </font>

In [9]:
# l is a matrix of [len(data),m*n] representing stochastic log-ratio errors         
L = np.zeros((len(df_fut),m*n)) #initialize empty array to populate below

# for the first five time periods, we set values to be the deterministic model lambdas across all realizations
# this is because we do not have enough time periods in the past to populate the AR4 model stochastically
for i in range(0,m*n):
    L[0:5,i] = data['lambda'].iloc[0:5] 

for i in range(0,m):    
    B = Beta[i,:] #let B = random parameter set for realization i
    for t in range(5,len(df_fut)):
        # L(t) = B0 + B1*L[t-1] + B2*L[t-2] + B3*L[t-3] + B4*L[t-4] + E[t]
        L[t, i*n:((i+1)*n)] = B[0] + \
                              B[1] * L[t-1, i*n:((i+1)*n)] + \
                              B[2] * L[t-2, i*n:((i+1)*n)] + \
                              B[3] * L[t-3, i*n:((i+1)*n)] + \
                              B[4] * L[t-4, i*n:((i+1)*n)] + \
                              E[t, i*n:((i+1)*n)]

### <font color = 'green'> &nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Step 3: Generate Stochastic Streamflows </font>

In [10]:
# Q is a matrix of stochastically simulated streamflows of size [m*n, len(data)]
# Just like lambda = log(Qmodel/Qgage), we now set Q = Qmodel/e^L where
# L is stochastic lambdas and Q is stochastic streamflow
Q = np.array(df_fut['Qfut']) / np.exp(L.T)

# BCF is transformation bias correction factor 
# this address the bias introduced by moving to log space
BCF = 1 / np.exp(-np.mean(data['lambda']) + np.var(data['lambda'])/2)

Q = Q*BCF

In [11]:
print(BCF)

0.9807641078478005


In [12]:
display(pd.DataFrame(Q))

,0,1,2,3,4,5,6,7,8,9,...,11313,11314,11315,11316,11317,11318,11319,11320,11321,11322
0,13.295533,11.020308,14.706603,15.334315,23.962067,30.554762,17.581626,74.290571,47.761557,55.977754,...,58.852985,69.462669,84.385785,108.348159,88.667614,71.520047,73.946177,93.677371,70.546656,97.914791
1,13.295533,11.020308,14.706603,15.334315,23.962067,14.307795,26.928484,32.168412,34.080161,30.915526,...,102.777906,72.012045,75.996002,72.706130,93.495066,70.454125,76.309652,87.553127,72.433736,93.495449
2,13.295533,11.020308,14.706603,15.334315,23.962067,29.049895,32.051125,31.638124,50.012314,42.097092,...,23.129647,64.260564,62.831425,58.309802,57.801281,110.331191,61.101082,64.106132,55.044516,85.923458
3,13.295533,11.020308,14.706603,15.334315,23.962067,19.835122,39.109741,36.634897,32.113311,49.056942,...,57.084464,66.912238,43.729328,89.511895,58.646963,36.023243,69.818490,67.235089,70.818965,80.908627
4,13.295533,11.020308,14.706603,15.334315,23.962067,26.207650,32.881136,44.583410,48.168007,54.797487,...,83.577702,106.485260,74.773849,89.756892,79.222277,75.111089,92.975075,72.166335,73.101387,125.947770
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,13.295533,11.020308,14.706603,15.334315,23.962067,32.775272,40.119558,37.682286,37.406073,41.522691,...,101.520242,81.874457,71.801375,86.210782,73.377577,68.882239,68.115391,60.930122,96.707783,101.116572
9996,13.295533,11.020308,14.706603,15.334315,23.962067,33.146049,27.096775,35.816441,48.099872,42.001052,...,37.343843,53.586659,61.351435,70.322242,74.176179,62.657614,63.832678,15.756751,81.557346,56.991275
9997,13.295533,11.020308,14.706603,15.334315,23.962067,40.453468,27.544222,37.438154,43.250700,79.496255,...,88.397129,88.787967,68.649338,94.562675,68.484809,85.446717,70.243825,70.356483,74.800908,102.086550
9998,13.295533,11.020308,14.706603,15.334315,23.962067,23.196166,31.806411,35.946256,40.093455,75.967863,...,67.805711,63.889471,52.700346,58.651553,56.149882,57.839838,77.376640,65.890201,91.790139,104.365404


In [13]:
display(pd.DataFrame(L.T))

,0,1,2,3,4,5,6,7,8,9,...,11313,11314,11315,11316,11317,11318,11319,11320,11321,11322
0,1.810035,1.996299,1.706123,1.66287,1.214848,0.970261,1.521397,0.079186,0.519693,0.359306,...,0.256758,0.087204,-0.110674,-0.363311,-0.165877,0.045579,0.008615,-0.231527,0.079155,-0.009154
1,1.810035,1.996299,1.706123,1.66287,1.214848,1.728977,1.095066,0.916185,0.857199,0.953001,...,-0.300769,0.051160,-0.005956,0.035613,-0.218891,0.060595,-0.022847,-0.163916,0.052757,0.037031
2,1.810035,1.996299,1.706123,1.66287,1.214848,1.020766,0.920919,0.932807,0.473645,0.644281,...,1.190686,0.165048,0.184269,0.256268,0.262007,-0.387930,0.199423,0.147790,0.327287,0.121486
3,1.810035,1.996299,1.706123,1.66287,1.214848,1.402327,0.721879,0.786169,0.916644,0.491278,...,0.287269,0.124612,0.546706,-0.172333,0.247482,0.731392,0.066053,0.100135,0.075303,0.181623
4,1.810035,1.996299,1.706123,1.66287,1.214848,1.123730,0.895352,0.589808,0.511219,0.380616,...,-0.093976,-0.340013,0.010256,-0.175066,-0.053240,-0.003412,-0.220379,0.029356,0.043582,-0.260924
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,1.810035,1.996299,1.706123,1.66287,1.214848,0.900107,0.696387,0.757980,0.764081,0.658020,...,-0.288457,-0.077193,0.050821,-0.134757,0.023399,0.083158,0.090749,0.198602,-0.236264,-0.041331
9996,1.810035,1.996299,1.706123,1.66287,1.214848,0.888858,1.088836,0.808763,0.512635,0.646565,...,0.711633,0.346694,0.208106,0.068950,0.012574,0.177871,0.155687,1.551061,-0.065877,0.532045
9997,1.810035,1.996299,1.706123,1.66287,1.214848,0.689629,1.072458,0.764480,0.618901,0.008550,...,-0.150039,-0.158257,0.095713,-0.227224,0.092406,-0.132337,0.059980,0.054755,0.020599,-0.050878
9998,1.810035,1.996299,1.706123,1.66287,1.214848,1.245794,0.928583,0.805145,0.694701,0.053949,...,0.115155,0.170839,0.360103,0.250425,0.290993,0.257879,-0.036733,0.120340,-0.184075,-0.072955


In [14]:
data

,date,Qgage,Qmodel,month,lambda,raw diff,Ar_res
0,1987-10-01,59.938142,366.261850,10,1.810035,306.323708,1.786596
1,1987-10-02,99.896904,735.416652,10,1.996299,635.519749,1.411201
2,1987-10-03,99.896904,550.188717,10,1.706123,450.291814,0.761985
3,1987-10-04,79.917523,421.519228,10,1.662870,341.601705,0.583921
4,1987-10-05,99.896904,336.630618,10,1.214848,236.733714,0.070037
...,...,...,...,...,...,...,...
11318,2018-09-26,79.917523,78.229589,9,-0.021347,-1.687934,-0.022115
11319,2018-09-27,79.917523,77.855393,9,-0.026142,-2.062129,-0.024599
11320,2018-09-28,59.938142,77.513991,9,0.257145,17.575849,0.261246
11321,2018-09-29,79.917523,81.235286,9,0.016355,1.317763,-0.024250


## <font color = 'green'> Save Outputs </font>

Outputs saved as **.npy** files. The outputs are:

1. E.npy: stochastic residuals
2. L.npy: stochastic log ratios
3. Q.npy: stochastic streamflows

In [15]:
# save the results as .npy files
np.save('Q_3degree.npy',Q)
np.save('StochasticE_3degree.npy',E)
np.save('L_3degree.npy',L)

***

# <center> <font color = 'red'> Part 2: Diagnostic Figures </font> </center>

Here, we include code to generate T-year flood flows for the specified warming scenario.

## <font color = 'green'> Reloading in the data </font>

In [16]:
# functions developed by Ghazal:
import Func_Lib_forAbby as func

# general packages
import numpy as np #numpy is a scientific computing package
import pandas as pd #pandas is a data analysis package
import math #math contains math functions
import time #functions in this package can be used to understand how long the code is taking to run
import os #functions related to operating system

# plotting packages
import matplotlib.pyplot as plt #matplotlib.pyplot is used for plotting
import seaborn as sns #seaborn is a plotting package
plt.style.use('seaborn-white') #this is the specific style of plotting used

# statistical analysis packages
from scipy import stats #scipy.stats are functions related to statistical analysis
from scipy.stats import norm #functions for normally distributed random variable 

# time series analysis packages
import statsmodels as sm
from statsmodels.graphics.tsaplots import plot_acf #function to plot autocorrelation function (acf) for time series analysis
from statsmodels.graphics.tsaplots import plot_pacf #function to plot partial autocorrelation function (pacf) for time series analysis
#from statsmodels.tsa.arima_model import ARIMA #import function for ARIMA (time series analysis)
from statsmodels.tsa.arima.model import ARIMA

In [17]:
# read in the csv of streamflow data and assign to variable called "data"
data = pd.read_csv('SWM-data_cond-debias.csv') #columns are date, Qgage (observed), Qmodel (deterministic model), and diff (Qmodel - Qgage)

data['month'] = pd.DatetimeIndex(data['date']).month #create a DatetimeIndex column for month
data['date'] = pd.to_datetime(data['date']) #convert format of "date" column to Datetime (for time series analysis)

#convert units to m^3/s 
# mm/day * SHA area in mi2 * (1609.34^2 m^2/mi^2) * (10^-3 m/1 mm) * (1 day/24*3600 s)
area_mi2_SHA = 6665 #area for Shasta baisn in sq. mi. from Zach Brodeur
data['Qgage'] = data['Qgage']*area_mi2_SHA*(1609.34**2)*(10**-3)/(24*3600)
data['Qmodel'] = data['Qmodel']*area_mi2_SHA*(1609.34**2)*(10**-3)/(24*3600)

# generating empirical log-ratio errors
data['lambda'] = np.log(data['Qmodel'] / data['Qgage']) #calculate the lambda = log(Qmodel/Qgage)
data['raw diff'] = data['Qmodel'] - data['Qgage']
data #display first five rows of data

,date,Qgage,Qmodel,month,lambda,raw diff
0,1987-10-01,59.938142,366.261850,10,1.810035,306.323708
1,1987-10-02,99.896904,735.416652,10,1.996299,635.519749
2,1987-10-03,99.896904,550.188717,10,1.706123,450.291814
3,1987-10-04,79.917523,421.519228,10,1.662870,341.601705
4,1987-10-05,99.896904,336.630618,10,1.214848,236.733714
...,...,...,...,...,...,...
11318,2018-09-26,79.917523,78.229589,9,-0.021347,-1.687934
11319,2018-09-27,79.917523,77.855393,9,-0.026142,-2.062129
11320,2018-09-28,59.938142,77.513991,9,0.257145,17.575849
11321,2018-09-29,79.917523,81.235286,9,0.016355,1.317763


In [18]:
#load in a "future" scenario into a new datafrae df_fut
df_fut = pd.read_csv('SWM-data_cond-debias_scenario-13.csv') #make sure this is one of the five we want, I just used a random one

#put the future scenario into the correct units of m3/s
df_fut['Qfut'] = df_fut['Qsim_cbias']*area_mi2_SHA*(1609.34**2)*(10**-3)/(24*3600)
df_fut['date'] = pd.to_datetime(df_fut['date'])
df_fut = df_fut[(df_fut['date'] > '1987-09-30') & (df_fut['date'] < '2018-10-01')]
df_fut = df_fut.reset_index()
df_fut['index']=np.arange(0, 11323)

#df_fut = df_fut.iloc[0:11323]

df_fut

,index,date,Qsim,Qsim_cbias,Qfut
0,0,1987-10-01,0.008264,0.414617,82.837995
1,1,1987-10-02,0.008134,0.414027,82.720013
2,2,1987-10-03,0.007987,0.413357,82.586194
3,3,1987-10-04,0.007855,0.412756,82.466052
4,4,1987-10-05,0.007708,0.412078,82.330604
...,...,...,...,...,...
11318,11318,2018-09-26,0.011616,0.382011,76.323410
11319,11319,2018-09-27,0.011326,0.380637,76.048830
11320,11320,2018-09-28,0.011037,0.379260,75.773864
11321,11321,2018-09-29,0.013276,0.389679,77.855357


In [19]:
Q = np.load('Q_3degree.npy')
m=100
n=100

In [20]:
pd.DataFrame(Q)

,0,1,2,3,4,5,6,7,8,9,...,11313,11314,11315,11316,11317,11318,11319,11320,11321,11322
0,13.295533,11.020308,14.706603,15.334315,23.962067,30.554762,17.581626,74.290571,47.761557,55.977754,...,58.852985,69.462669,84.385785,108.348159,88.667614,71.520047,73.946177,93.677371,70.546656,97.914791
1,13.295533,11.020308,14.706603,15.334315,23.962067,14.307795,26.928484,32.168412,34.080161,30.915526,...,102.777906,72.012045,75.996002,72.706130,93.495066,70.454125,76.309652,87.553127,72.433736,93.495449
2,13.295533,11.020308,14.706603,15.334315,23.962067,29.049895,32.051125,31.638124,50.012314,42.097092,...,23.129647,64.260564,62.831425,58.309802,57.801281,110.331191,61.101082,64.106132,55.044516,85.923458
3,13.295533,11.020308,14.706603,15.334315,23.962067,19.835122,39.109741,36.634897,32.113311,49.056942,...,57.084464,66.912238,43.729328,89.511895,58.646963,36.023243,69.818490,67.235089,70.818965,80.908627
4,13.295533,11.020308,14.706603,15.334315,23.962067,26.207650,32.881136,44.583410,48.168007,54.797487,...,83.577702,106.485260,74.773849,89.756892,79.222277,75.111089,92.975075,72.166335,73.101387,125.947770
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,13.295533,11.020308,14.706603,15.334315,23.962067,32.775272,40.119558,37.682286,37.406073,41.522691,...,101.520242,81.874457,71.801375,86.210782,73.377577,68.882239,68.115391,60.930122,96.707783,101.116572
9996,13.295533,11.020308,14.706603,15.334315,23.962067,33.146049,27.096775,35.816441,48.099872,42.001052,...,37.343843,53.586659,61.351435,70.322242,74.176179,62.657614,63.832678,15.756751,81.557346,56.991275
9997,13.295533,11.020308,14.706603,15.334315,23.962067,40.453468,27.544222,37.438154,43.250700,79.496255,...,88.397129,88.787967,68.649338,94.562675,68.484809,85.446717,70.243825,70.356483,74.800908,102.086550
9998,13.295533,11.020308,14.706603,15.334315,23.962067,23.196166,31.806411,35.946256,40.093455,75.967863,...,67.805711,63.889471,52.700346,58.651553,56.149882,57.839838,77.376640,65.890201,91.790139,104.365404


## <font color = 'green'> Generating Figure Inputs </font>

In [21]:
def annualmax(Q,data,n):
    annual_max=np.array(pd.DataFrame(Q.T,index=data['date']).resample('Y').max())
    annualmaxsort = np.zeros(annual_max.shape)
    for i in range(0,n):
        annualmaxsort[:,i]=np.sort(annual_max[:,i])[::-1]
    
    annualmaxsort = pd.DataFrame(annualmaxsort)
    #annualmaxsort['rank']=(annualmaxsort.index) + 1
    #annualmaxsort['Exeedance']=annualmaxsort['rank']/(len(annualmaxsort)+1)
    #annualmaxsort['non_Exceedance']=1-annualmaxsort['Exeedance']
    #Data=data.set_index('date')
    #annualmaxobs=Data['Qgage'].resample('Y').max()
    #annualmaxmodel=Data['Qmodel'].resample('Y').max()
    #annualmaxsort['obs']=np.sort(annualmaxobs)[::-1]
    #annualmaxsort['model']=np.sort(annualmaxmodel)[::-1]
    amax=np.array(annualmaxsort)[:,0:n]
    return annualmaxsort
Annualmax = annualmax(Q, df_fut, m*n)

In [22]:
# calculating 50-year floods
from scipy.stats import pearson3
ann_max=np.array(Annualmax)
P=1-(1/50)
flood50Y3=np.exp(np.log(pd.DataFrame(ann_max[:,0:10000])).mean()+np.log(pd.DataFrame((ann_max[:,0:10000]))).std()*abs(pearson3.ppf(P, np.log(pd.DataFrame(ann_max[:,0:10000])).skew())))
flood50Y3 = np.array(flood50Y3)

In [23]:
# calculating 100-year floods
from scipy.stats import pearson3
ann_max=np.array(Annualmax)
P=1-(1/100)
flood100Y3=np.exp(np.log(pd.DataFrame(ann_max[:,0:10000])).mean()+np.log(pd.DataFrame((ann_max[:,0:10000]))).std()*abs(pearson3.ppf(P, np.log(pd.DataFrame(ann_max[:,0:10000])).skew())))
flood100Y3 = np.array(flood100Y3)

In [24]:
# calculating 500-year floods
from scipy.stats import pearson3
ann_max=np.array(Annualmax)
P=1-(1/500)
flood500Y3=np.exp(np.log(pd.DataFrame(ann_max[:,0:10000])).mean()+np.log(pd.DataFrame((ann_max[:,0:10000]))).std()*abs(pearson3.ppf(P, np.log(pd.DataFrame(ann_max[:,0:10000])).skew())))
flood500Y3 = np.array(flood500Y3)

In [25]:
np.save('flood50Y3.npy',flood50Y3)
np.save('flood100Y3.npy',flood100Y3)
np.save('flood500Y3.npy',flood500Y3)